In [ ]:
# easier way to show things.
# the first notebook was a bit confusing or hard to read from official LangGraph doc pages.

In [ ]:
from dotenv import load_dotenv
import os

# 1st method: using .env file.
load_dotenv()
# Access them using os.getenv or os.environ
api_key = os.getenv("GROQ_API_KEY")

# 2nd method: using hard code
# api_key = "<put the api key here>"
# if not os.environ.get("GROQ_API_KEY"):
#     os.environ["GROQ_API_KEY"] = api_key #getpass.getpass("Enter API key for Groq: ")




In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")
# llm2 = ChatGroq(model="llama3-8b-8192")

In [ ]:
from langgraph.graph.message import add_messages
from langchain.chat_models import init_chat_model
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph import START, END,  MessageGraph


In [ ]:
graph = MessageGraph() 

In [ ]:
# llm1.invoke([SystemMessage(content="You are a creative thinker. Come up with wild, imaginative ideas without worrying about realism."), 
#              HumanMessage(content="Explain quantum computing to a 12-year-old.")])

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

generation_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a Twitter writer."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

reflect_prompt = ChatPromptTemplate.from_template(
    """You are reviewing the following tweet:
"{messages}"

If it's already good, return it as-is. Otherwise, revise it to improve clarity, tone, or engagement."""
)

generation_chain = generation_prompt | llm
reflect_chain = reflect_prompt | llm

In [ ]:
from langchain_core.prompts import ChatPromptTemplate



In [ ]:
def generate_node(msg):
    print('\n generate node:\n')
    return generation_chain.invoke({
    "messages":msg
    })

def reflect_node(msg):
    print('\n reflect node:\n')
    return reflect_chain.invoke({
    "messages":msg
    })

In [ ]:
graph.add_node("generate", generate_node)
graph.add_node("reflect", reflect_node)

In [ ]:
def should_continue(msg):
    # print('msg[generate node]=',msg["generate_node"])
    print('len(msg)=', len(msg))
    if (len(msg)>3):
        return "to_end"
    return "to_reflect"

In [ ]:
# graph.add_edge(START, "generate")
graph.set_entry_point("generate")

graph.add_conditional_edges("generate", 
                            should_continue,
                            
                          {    
        "to_reflect": "reflect",  # Mapping function output to nodes
        "to_end": END,
    },     
                           )
graph.add_edge("reflect", "generate")

# graph.add_edge("generate", "reflect")


# graph.set_conditional_entry_point("generate")
# graph.add_edge("generate", "reflect")

In [ ]:
app = graph.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception:
    
    # This requires some extra dependencies and is optional
    pass

In [ ]:
print(app.get_graph().print_ascii())

In [ ]:
# !pip install grandalf

In [ ]:
content = app.invoke(HumanMessage(content="make a tweet on Sunny days"))

In [ ]:
content

In [ ]:
graph.edges

In [ ]:
print("\n[Final Messages]")
for msg in content:
    print(f"- {type(msg).__name__}: {msg.content}")